In [ ]:
"""danielsinkin97@gmail.com"""

import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, FloatSlider

from src.filter import FilterType, apply_filter, get_filter
from util.image_to_np import load_image_as_array
from util.rbg_to_grayscale import rgb_to_grayscale

loaded_image = rgb_to_grayscale(
    load_image_as_array(
        "/Users/danielsinkin/GitHub_private/computer-vision/data/hummingbird.png"
    )
)

i_h, i_w = loaded_image.shape

filter_ = np.array([-2, -1, 0, 1, 2], np.float32)
i_x = apply_filter(loaded_image, filter_, pad_same_size=True)

vertical_filter = filter_.reshape(-1, 1)
i_y = apply_filter(loaded_image, vertical_filter, pad_same_size=True)

w = get_filter(FilterType.GAUSS_5X5)

i_xx = i_x * i_x
i_xy = i_x * i_y
i_yy = i_y * i_y

a11 = apply_filter(i_xx, w, pad_same_size=True)
a12 = apply_filter(i_xy, w, pad_same_size=True)
a22 = apply_filter(i_yy, w, pad_same_size=True)

A = np.stack([np.stack([a11, a12], axis=-1), np.stack([a12, a22], axis=-1)], axis=-2)
assert A.shape == (i_h, i_w, 2, 2)

delta_u = np.array([10, -5], dtype=np.float32)
E = np.einsum("i, ...ij, j -> ...", delta_u, A, delta_u)

interest_highlighting = apply_filter(
    E, get_filter(FilterType.GAUSS_15X15), pad_same_size=True
)
interest_highlighting = (
    (interest_highlighting - interest_highlighting.min())
    / (interest_highlighting.max() - interest_highlighting.min())
    * 255
)

@interact(d=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.4, description="Blend d"))
def update(d=0.4):
    blended = d * loaded_image + (1 - d) * interest_highlighting

    fig, axs = plt.subplots(1, 3, figsize=(12, 8))

    axs[0].imshow(blended, cmap="viridis", interpolation="nearest")
    axs[0].set_title("Viridis", color="white")
    axs[0].axis("off")

    axs[1].imshow(blended, cmap="magma", interpolation="nearest")
    axs[1].set_title("Magma", color="white")
    axs[1].axis("off")

    axs[2].imshow(blended, cmap="gray", interpolation="nearest")
    axs[2].set_title("gray", color="white")
    axs[2].axis("off")

    fig.tight_layout()
    fig.patch.set_facecolor("black")  # dark background
    plt.show()